# Tutorial for the neural representation of visually evoked emotion is high-dimensional, categorical, and distributed across transmodal brain regions

This notebook provides a tutorial for our paper:  [Horikawa, Cowen, Keltner, and Kamitani (2020) The neural representation of visually evoked emotion is high-dimensional, categorical, and distributed across transmodal brain regions. iScience](https://www.cell.com/iscience/fulltext/S2589-0042(20)30245-5?rss=yes).


このチュートリアルでは， 下記の github リポジトリで decoding および encoding analysis を行うための手続きを確認していきます．

- [KamitaniLab/EmotionVideoNeuralRepresentationPython](https://github.com/KamitaniLab/EmotionVideoNeuralRepresentationPython)

Decoding および encoding のアルゴリズムの詳細を学びたい場合は，下記の別のリポジトリを参照してください．

- [KamitaniLab/feature-decoding](https://github.com/KamitaniLab/feature-decoding)
- [KamitaniLab/feature-encoding](https://github.com/KamitaniLab/feature-encoding)


なお，ここでの計算はあくまでチュートリアル向けであり， 上記のpaperにある全ての計算は行っていないことに注意してください．paperの条件で計算を行う場合は，チュートリアル用のconfig fileではなく解析用のconfig fileを使用して下さい．

## Environment setup
このチュートリアルはGoogle colabでも実行でき，基本的にはGoogle colabのコマンドおよびマジックコマンドで手続きが示されています．

** ローカルマシン上で実行する場合，このSetup セクションのノートブックセルは全て実行する必要はありません．** セル上に示されているコマンドライン操作を，各マシンのCUI上で実行してください．

### ディレクトリ構成の準備
下記のgithubリポジトリをcloneしてください．


In [ ]:
# ローカルマシンで実行している場合，httpsがセキュリティ的に実行できない場合があります．その場合は，以下を clone 元として使用してください．
# git@github.com:KamitaniLab/EmotionVideoNeuralRepresentationPython.git 
!git clone https://github.com/KamitaniLab/EmotionVideoNeuralRepresentationPython.git
%cd EmotionVideoNeuralRepresentationPython
!ls

ディレクトリEmotionVideoNeuralRepresentationPythonの下に，さらに下記の2つのgithubリポジトリをcloneしてください．
これらのスクリプトを使用することが前提になっています．

In [ ]:
# ローカルマシンで実行している場合，httpsがセキュリティ的に実行できない場合があります．その場合は，以下を clone 元として使用してください．
# git@github.com:KamitaniLab/feature-decoding.git
# git@github.com:KamitaniLab/feature-encoding.git
!git clone https://github.com/KamitaniLab/feature-decoding.git
!git clone https://github.com/KamitaniLab/feature-encoding.git

ディレクトリ構成が下記のようになっていることを確認してください．

```
EmotionVideoNeuralRepresentationPython (current directory)
├── README.md
├── config/
├── data/
├── env.yaml
├── feature-decoding/
├── feature-encoding/
├── requirements.txt
└── src/
```

In [ ]:
!pwd
!ls -1

### 必要なライブラリのインストールと設定の修正
EmotionVideoNeuralRepresentationPython 内の環境設定ファイルを用いて，環境の構築を行います．これにより，追加でcloneした2つのリポジトリも動作する環境が構築されます．

ローカルマシンで実行している場合は，下記のセルを実行する代わりに， 下記のコマンドを実行し， conda 環境を作成することを推奨します． 
```
$ conda env create --name <your environment name> -f env.yaml
$ activate <your environment name>
```

In [ ]:
!pip install -r ./requirements.txt

[pycortex](https://github.com/gallantlab/pycortex)を使用した皮質マップの可視化を行う場合，[Inkscape](https://wiki.inkscape.org/wiki/Installing_Inkscape)をインストールしてください．
Google colabの場合は下記のセルを実行し，ローカルマシンで実行している場合は使用している環境に合わせて適宜インストールしてください．


In [ ]:
!sudo apt-get install -y inkscape

pycortex での可視化を行う場合は， 設定ファイルを書き換えます．このチュートリアルをローカルマシンで実行している場合も，同様に書き換えてください．

`~/.config/pycortex/options.cfg` の `[base]` セクションの `filestore` のパスを `data/pycortex` 以下に修正します．もし，誤ってすでに `import cortex` してしまった場合は，ランタイムを再起動してください．

In [ ]:
!cat  ~/.config/pycortex/options.cfg | grep filestore
!cp ~/.config/pycortex/options.cfg ~/.config/pycortex/backup_options.cfg
!sed -i 's/filestore = \/usr\/share\/pycortex\/db/filestore = .\/data\/pycortex\//' ~/.config/pycortex/options.cfg
!cat  ~/.config/pycortex/options.cfg | grep filestore


### 必要なデータのダウンロード

dataディレクトリの下に移動し， download.pyを実行して下さい．
download.py はダウンロード対象のデータを引数で指定することができます．今回はデモに使用するファイルのみをダウンロードします．

- demo_fmri: Subject1のfMRIデータ
- demo_pycortex: Subject1のpycotexデータ
- feautres: 全てのemotional scores

終了後は， `data`を抜けて，`EmotionVideoNeuralRepresentationPython`の直下に移動してください．

In [ ]:
%cd data
!python download.py demo_fmri
!python download.py demo_pycortex
!python download.py features
%cd ../

これで準備は完了です．


## Tutorial for encoding analysis

このセクションではencoding analysisの手続きについて説明します．このリポジトリでは，下記の手順でencodingを実行してきます．さらに次のセクションで説明するdecoding analysisについてもほぼ同じ手順で実行できます．
1. Encoding 設定ファイルを確認する
2. Encoderのtrainingを行う
3. Encoding prediction を行う
4. Encoding prediction の performance の計算
5. Encoding performance の可視化 



### Encoding 設定ファイルの確認

まずは必要なモジュールをimportしてください

In [ ]:
import os
import sys
from pathlib import Path
from hydra.experimental import initialize_config_dir, compose
from pathlib import Path
from omegaconf import OmegaConf, DictConfig


今回の計算対象の確認を行います．

本稿では， encoding anlaysis として， emotional category (34 category) および affective dimension (14 dimension) の score を用いて，各被験者の皮質脳活動を推定しています．

実際には5人の被験者に対して，それぞれ6-fold crossvalidationの計算を行っています．しかし，すべての計算を行うと時間がかかるため，ここでは省略のため，下記の条件に絞り込みます．

1. 5subjectのうち，1subjectの計算のみを行う
2. semantic scoreおよびvision scoreの計算を行わず，category score, dimension scoreの計算のみを行う
2. 6-fold計算のうち，1-foldの計算のみを行う
3. fold計算中に，さらに入れ子のfold計算を行い，パラメータの最適化を行うフェーズが存在しない （これは現在のPythonコードリポジトリでは実装されていない機能です． オリジナルの[Matlabコードのリポジトリ](https://github.com/KamitaniLab/EmotionVideoNeuralRepresentation/tree/master?tab=readme-ov-file)をご参照ください．）

それでは，今回使用する tutorial 用の config ファイル `./config/tutorial_encoding_emotion_score_cv_paper2020.yaml` を確認しましょう．


In [ ]:
# ここで変数状態にまで持っていく

config_file = "./config/tutorial_encoding_emotion_score_cv_paper2020.yaml"
config_file = Path(config_file)
config_name = str(config_file.stem)
config_dir = str(config_file.absolute().parent)

with initialize_config_dir(config_dir=config_dir):
    cfg = compose(config_name=config_name)    


# Convert settings
analysis_name = 'cv_train_encoder_fastl2lir-tutorial_encoding_emotion_score_cv_paper2020'
fmri = {
    subject["name"]: subject["paths"]
    for subject in cfg["encoder"]["fmri"]["subjects"]
}
rois = {
    roi["name"]: roi["select"]
    for roi in cfg["encoder"]["fmri"]["rois"]
}
label_key = cfg["encoder"]["fmri"]["label_key"]
feature_dir_list = cfg["encoder"]["features"]["paths"]
features = [feat["name"] for feat in cfg["encoder"]["features"]["layers"]]
num_unit = {
    layer["name"]: layer["num"]
    for layer in cfg["encoder"]["features"]["layers"]
}
feature_index_file = cfg.encoder.features.get("index_file", None)
encoder_dir = cfg["encoder"]["path"]
encoded_fmri_dir = cfg["encoded_fmri"]["path"]
cv_folds = cfg.cv.get("folds", None)
cv_exclusive = cfg.cv.get("exclusive_key", None)
cv_labels = ['cv-fold{}'.format(icv + 1) for icv in range(len(cv_folds))]
excluded_labels = cfg.encoded_fmri.fmri.get("exclude_labels", [])
average_sample = cfg.encoded_fmri.parameters.get("average_sample", True)
evaluation_rois = {
    roi["name"]: roi["select"]
    for roi in cfg["evaluation"]["fmri"]["rois"]
}

print("====== Encoding settings ======")
print("Subjects: {}".format(list(fmri.keys())))
print("ROIs:     {}".format(list(rois.keys())))
print("Features: {}".format(features))
print("Folds:    {}".format(cv_labels))
print("")
print("fMRI data dir:           {}".format(os.path.split(list(fmri.values())[0][0])[0]))
print("Feature data dir:        {}".format(feature_dir_list[0]))
print("")
print("Save encoder dir:        {}".format(encoder_dir))
print("Save encoded singal dir: {}".format(encoded_fmri_dir))
print("===============================")


上記の条件で計算を行っていきます．実際の解析で用いられている条件についても，下記のセルを実行すると確認できます．

config file: `./config/encoding_emotion_score_cv_paper2020.yaml` 

すべての条件を計算する場合は，計算量の増加により計算時間がスケールすることに注意してください．（ある程度のスペックのあるローカルマシン環境で計算することを推奨します．）

In [ ]:

# Original encoding analysis settings

config_file = "./config/encoding_emotion_score_cv_paper2020.yaml"
config_file = Path(config_file)
config_name = str(config_file.stem)
config_dir = str(config_file.absolute().parent)

with initialize_config_dir(config_dir=config_dir):
    cfg2 = compose(config_name=config_name)    

# Convert settings
fmri2 = {
    subject["name"]: subject["paths"]
    for subject in cfg2["encoder"]["fmri"]["subjects"]
}
rois2 = {
    roi["name"]: roi["select"]
    for roi in cfg2["encoder"]["fmri"]["rois"]
}
feature_dir_list2 = cfg2["encoder"]["features"]["paths"]
features2 = [feat["name"] for feat in cfg2["encoder"]["features"]["layers"]]
encoder_dir2 = cfg2["encoder"]["path"]
encoded_fmri_dir2 = cfg2["encoded_fmri"]["path"]
cv_folds2 = cfg2.cv.get("folds", None)
cv_labels2 = ['cv-fold{}'.format(icv + 1) for icv in range(len(cv_folds2))]

from pprint import pprint
print("====== Encoding settings ======")
print("Subjects: {}".format(list(fmri2.keys())))
print("ROIs:     {}".format(list(rois2.keys())))
print("Features: {}".format(features2))
print("Folds:    {}".format(cv_labels2))
print("")
print("fMRI data dir:           {}".format(os.path.split(list(fmri2.values())[0][0])[0]))
print("Feature data dir:        {}".format(feature_dir_list2[0]))
print("")
print("Save encoder dir:        {}".format(encoder_dir2))
print("Save encoded singal dir: {}".format(encoded_fmri_dir2))
print("===============================")

今回の計算対象の確認を行います．

本稿では， encoding anlaysis として， emotional category (34 category) および affective dimension (14 dimension) の score を用いて，各被験者の皮質脳活動を推定しています．

実際には5人の被験者に対して，それぞれ6-fold crossvalidationの計算を行っています．しかし，すべての計算を行うと時間がかかるため，ここでは省略のため，下記の条件に絞り込みます．

1. 5subjectのうち，1subjectの計算のみを行う
2. semantic scoreおよびvision scoreの計算を行わず，category score, dimension scoreの計算のみを行う
2. 6-fold計算のうち，1-foldの計算のみを行う
3. fold計算中に，さらに入れ子のfold計算を行い，パラメータの最適化を行うフェーズが存在しない （これは現在のPythonコードリポジトリでは実装されていない機能です． オリジナルの[Matlabコードのリポジトリ](https://github.com/KamitaniLab/EmotionVideoNeuralRepresentation/tree/master?tab=readme-ov-file)をご参照ください．）

それでは，今回使用する tutorial 用の config ファイル `./config/tutorial_encoding_emotion_score_cv_paper2020.yaml` を確認しましょう．


それでは，上記のtutorial のセッティングで計算を行っていきます．

### Encoder training を行う (~5分)
Encoderのtrainingを行います． 実行時のカレントディレクトリが， `EmotionVideoNeuralRepresentationPython` の直下であることを確認してください．

もし，ローカル環境で実行している場合，下記の通りコマンドを実行することで，上記の設定ファイルに準じて encoder の training が開始されます

`$ python feature-encoding/cv_train_encoder_fastl2lir.py config/encoding_emotion_score_cv_paper2020.yaml`

この計算は，異なるマシン上で並行して実行することが可能です（カレントディレクトリ，および保存先ディレクトリをマシン間で共有している必要があります）．
特に，全ての条件を計算する場合，複数マシンでの並列実行を推奨します．

ここでは，　上記のスクリプト `cv_train_encoder_fastl2lir.py` で呼び出されるメイン関数を直接呼び出し，実行します．

In [ ]:
sys.path.insert(0, "feature-encoding")
from cv_train_encoder_fastl2lir import featenc_cv_fastl2lir_train

featenc_cv_fastl2lir_train(
    fmri,
    feature_dir_list,
    output_dir=encoder_dir,
    rois=rois,
    label_key=label_key,
    layers=features,
    num_unit=num_unit,
    feature_index_file=feature_index_file,
    alpha=cfg["encoder"]["parameters"]["alpha"],
    cv_key=cfg["cv"]["key"],
    cv_folds=cv_folds,
    cv_exclusive=cv_exclusive,
    analysis_name=analysis_name
)


下記のディレクトリに訓練済みencoderモデルが出力されます．

`<Save encoder dir>/<feature>/<subject>/<ROI>/<Fold>/model`

- `W.mat`: Weight of encoder model
- `b.mat`: Bias of encoder model
- `x_mean.mat`, `x_norm.mat`: Feature normalization parameters. 
- `y_mean.mat`, `y_norm.mat`: Voxel normalization parameters.

In [ ]:
!ls ./data/feature_encoders/encoding_emotion_score_cv_paper2020_featall_alpha100/amt/mean_score_concat
!ls ./data/feature_encoders/encoding_emotion_score_cv_paper2020_featall_alpha100/amt/mean_score_concat/category/Subject1/WholeBrain/cv-fold1/model


計算終了後， 下記のスクリプトを実行することで， 全ての計算が完了していることを確認します．並列計算した場合は，必ず実行してください．

`$ python src/check_training_prediction.py config/encoding_emotion_score_cv_paper2020.yaml --override +wash=train`

ここでは，　上記のスクリプト `` で呼び出されるメイン関数を直接呼び出し，実行します．
下記が出力されれば問題ありません．

```
==========
No processes were terminated during execution.
==========
```

もし，　未終了の条件があると指示された場合は，再度，上記のセル（コマンド）を実行し，すべての条件を確実に完了させてください．

In [ ]:
sys.path.insert(0, "src")
from check_training_prediction import check_training_prediction

target = "train"
analysis_type = "encoding"

check_training_prediction(
    encoder_dir,
    target,
    analysis_type,
    analysis_name,
    list(fmri.keys()),
    list(rois.keys()),
    features,
    cv_folds,
)


これで encoder の training は完了です．
Encoding(training/prediction)の詳細な手続きを知りたい場合は，　feature-encoding リポジトリ内の handson 02_encoding.ipynb を参照してください．

### Encoding prediction を行う（~5分）
前セクションでtrainingしたencoderを用いて，predictionを行います．実行時のカレントディレクトリは， `EmotionVideoNeuralRepresentationPython` の直下としてください．

もし，ローカル環境で実行している場合，下記の通りコマンドを実行することで，上記の設定ファイルに準じて encoder の prediction が開始されます

`$ python src/cv_predict_fmri_fastl2lir.py config/encoding_emotion_score_cv_paper2020.yaml`

この計算も，並列実行することが可能です．

ここでは，　上記のスクリプト `cv_predict_fmri_fastl2lir.py` で呼び出されるメイン関数を直接呼び出し，実行します．

In [ ]:
sys.path.insert(0, "feature-encoding")
from cv_predict_fmri_fastl2lir import featenc_cv_fastl2lir_predict

featenc_cv_fastl2lir_predict(
    feature_dir_list,
    fmri,
    encoder_dir,
    output_dir=encoded_fmri_dir,
    subjects=list(fmri.keys()),
    rois=rois,
    label_key=label_key,
    layers=features,
    feature_index_file=feature_index_file,
    excluded_labels=excluded_labels,
    cv_key=cfg["cv"]["key"],
    cv_folds=cv_folds,
    cv_exclusive=cv_exclusive,
    analysis_name=analysis_name
)



計算終了後， 下記のスクリプトを実行することで， 全ての計算が完了していることを確認します．

`$ python src/check_training_prediction.py config/encoding_emotion_score_cv_paper2020.yaml --override +wash=predict`

ここでは，　上記のスクリプトで呼び出されるメイン関数を直接呼び出し，実行します．下記の2つの結果が出力されることを確認してください．
```
==========
No processes were terminated during execution.
==========

==========
All predicted files exist.
==========
```
もし，未終了の条件があると指示された場合は，再度，上記のセル（コマンド）を実行し，すべての条件を確実に完了させてください．


In [ ]:
sys.path.insert(0, "src")
from check_training_prediction import check_training_prediction

target = "predict"
analysis_type = "encoding"

check_training_prediction(
    encoded_fmri_dir,
    target,
    analysis_type,
    analysis_name,
    list(fmri.keys()),
    list(rois.keys()),
    features,
    cv_folds,
)


これで encoder の prediction は完了です．
Encoding(training/prediction)の詳細な手続きを知りたい場合は，　feature-encoding リポジトリ内の handson 02_encoding.ipynb を参照してください．

### Encoding prediction の performance の計算 （~5分）

Encoding結果の評価を行います．具体的には，前セクションで推定されたfMRI signalと，実際に観測されたfMRI singalについて，voxelごとにcorrelation coefficientを計算します．
実行時のカレントディレクトリは， EmotionVideoNeuralRepresentationPython の直下としてください．


もし，ローカル環境で実行している場合，下記の通りコマンドを実行することで，上記の設定ファイルに準じて encoder の training が開始されます

`$ python src/cv_evaluate_predicted_fmri.py config/encoding_emotion_score_cv_paper2020.yaml `

この計算は，並列実行することが可能です．デモでは不要ですが，全ての条件を計算する場合，複数台のマシンでの並列実行を推奨します．

ここでは，　上記のスクリプト `cv_evaluate_predicted_fmri.py` で呼び出されるメイン関数を直接呼び出し，実行します．

In [ ]:
from cv_evaluate_predicted_fmri import cv_evaluate_predicted_fmri

cv_evaluate_predicted_fmri(
    encoded_fmri_dir,
    encoder_dir,
    fmri,
    rois,
    evaluation_rois,
    features,
    cv_folds,
    label_key,
    output_file_pooled=os.path.join(encoded_fmri_dir, 'evaluation.db'),
    output_file_fold=os.path.join(encoded_fmri_dir, 'evaluation_fold.db'),
    average_sample=average_sample,
)


計算終了後， 下記のスクリプトを実行することで， 全ての計算が完了していることを確認します．

`$ python src/check_evaluate_database.py config/encoding_emotion_score_cv_paper2020.yaml `

ここでは，　上記のスクリプトで呼び出されるメイン関数を直接呼び出し，実行します． `All conditions are finished` が二度出力されていることを確認してください．一つは各foldの計算が完了していることを示しており，もう一つはfoldの計算結果を平均したpooledの計算が完了していることを示しています．
```
==========
All conditions are finished.
==========
```
もし，　未終了の条件があると指示された場合は，再度，上記のセル（コマンド）を実行し，すべての条件を確実に完了させてください．

In [ ]:
sys.path.insert(0, "src")
from check_evaluate_database import check_evaluate_database

analysis_type = "encoding"

check_evaluate_database(
    encoded_fmri_dir,
    analysis_type,
    list(fmri.keys()),
    list(rois.keys()),
    features,
    cv_folds,
    output_file_pooled=os.path.join(encoded_fmri_dir, 'evaluation.db'),
    output_file_fold=os.path.join(encoded_fmri_dir, 'evaluation_fold.db'),
)


これで encoding analysis の全てのステップが完了しました．
Encoding(training/prediction/evaluation)の詳細な手続きを知りたい場合は，　[feature-encoding](https://github.com/KamitaniLab/feature-encoding) リポジトリ内の handson 02_encoding.ipynb を参照してください．

### Encoding performance の可視化 （~5分）

ここでは前セクションで計算された correlation coefficient を，皮質上にマップして結果を確認します．


Pycortexを使用する場合， `pycortex` の `filestore` が `./data/pycortex` であること，および`INKSCAPE_VERSION`が`None`ではないことを確認してください．（条件が満たされていない場合，ランタイムを再起動させてください．）

In [ ]:
import cortex
print('pycortex db:', cortex.config['basic']['filestore'])
print('INKSCAPE_VERSION:', cortex.testing_utils.INKSCAPE_VERSION)



下記のセルを実行して，結果の可視化を行ってください．各被験者のemotion scoreごとのencoding accuracyが表示されます．

In [ ]:
import bdpy
from bdpy.dataform import SQLite3KeyValueStore
from itertools import product
import numpy as np
import matplotlib.pyplot as plt

pycortex_align = 'voxel2mm_fmriprep'
pycortex_overlay_filepath = './data/pycortex/{}/overlays.svg'
metric = "profile_correlation"
vmin = 0.1
vmax = 0.4

for subject, roi, evaluation_roi in product(fmri.keys(), rois.keys(), evaluation_rois.keys()):
    # Load Bdata and voxel information
    bdata = bdpy.BData(fmri[subject][0])
    _, encoding_roi_selector = bdata.select(roi, return_index = True)
    _, visualize_roi_selector = bdata.select(evaluation_roi, return_index = True)
    voxel_selector = visualize_roi_selector[encoding_roi_selector]
    
    # Load accuracy
    evaluation_filepath = os.path.join(encoded_fmri_dir, "evaluation.db")
    db = SQLite3KeyValueStore(evaluation_filepath)
        
    for feature in features:
        print("Feature:", feature)

        # Create emty volume
        # Colormap (cmap) for visualization is set here. Available cmap is same as Matplotlib 
        # (See document for matplotlib.pylab.colormaps()).
        voxel_vol = cortex.Volume.empty(subject, 
                                        pycortex_align, 
                                        value=np.nan, cmap='hot')                

        # Apply voxel selector to voxel_i/j/k array
        voxel_ijk = np.vstack([
            bdata.get_metadata('voxel_i')[visualize_roi_selector].astype('int32'),
            bdata.get_metadata('voxel_j')[visualize_roi_selector].astype('int32'),
            bdata.get_metadata('voxel_k')[visualize_roi_selector].astype('int32'),
        ]).astype(np.int)

        # Get each voxel accuracy
        acc = db.get(subject=subject, roi=roi, layer=feature, metric=metric)
        acc = acc[voxel_selector]
        acc[acc < vmin] = np.nan
        voxel_vol.data[ voxel_ijk[2], voxel_ijk[1], voxel_ijk[0] ] = acc

        # Set vmax and vmin for visualization                    
        voxel_vol.vmax = vmax
        voxel_vol.vmin  = 0

        # Set text
        text_posi_x = 0.5
        text_posi_y = 0.9
        show_text = feature
        
        # Show
        fig = cortex.quickshow( 
                    voxel_vol, with_colorbar=True, with_curvature=True, 
                    overlay_file=pycortex_overlay_filepath.format(subject),
                    labelsize=50, linewidth=3,
            )
        fig.text(text_posi_x, text_posi_y, show_text, 
                    horizontalalignment='center', verticalalignment='center', size=50)
        plt.show()
        

以上でencoding analysisのtutorialは終了です．


## Tutorial for decoding analysis

### Decoding 設定ファイルの確認

In [ ]:
# ここで変数状態にまで持っていく

config_file = "./config/tutorial_decoding_emotion_score_cv_paper2020.yaml"
config_file = Path(config_file)
config_name = str(config_file.stem)
config_dir = str(config_file.absolute().parent)

with initialize_config_dir(config_dir=config_dir):
    cfg = compose(config_name=config_name)
    
training_fmri = {
    subject["name"]: subject["paths"]
    for subject in cfg["decoder"]["fmri"]["subjects"]
}
rois = {
    roi["name"]: roi["select"]
    for roi in cfg["decoder"]["fmri"]["rois"]
}
num_voxel = {
    roi["name"]: roi["num"]
    for roi in cfg["decoder"]["fmri"]["rois"]
}
label_key = cfg["decoder"]["fmri"]["label_key"]
training_features = cfg["decoder"]["features"]["paths"]
features = cfg["decoder"]["features"]["layers"]
feature_index_file = cfg.decoder.features.get("index_file", None)
decoder_dir = cfg["decoder"]["path"]
cv_folds = cfg.cv.get("folds", None)
cv_exclusive = cfg.cv.get("exclusive_key", None)

if 'name' in cv_folds[0]:
    cv_labels = ['cv-{}'.format(cv['name']) for cv in cv_folds]
else:
    cv_labels = ['cv-fold{}'.format(icv + 1) for icv in range(len(cv_folds))]

from pprint import pprint
print("====== Encoding settings ======")
print("Subjects(fMRI data):", [subject["name"] for subject in cfg["decoder"]["fmri"]["subjects"]])
print("ROIs:", [roi["name"] for roi in cfg["decoder"]["fmri"]["rois"]])
print("Features:", cfg["decoder"]["features"]["layers"])
print("Folds:", cv_folds)
print("")


### Decoder trainingを行う（~5分）